In [1]:
import json
from pathlib import Path
import os
import re

In [2]:
with open("/workdir/triton-seeds/command_mapping.json", "r") as f:
    cmd_string_mapping = json.load(f)

In [3]:
def get_cmd(text):
    # for now let's only consider mlir-opt
    pattern = r"^(triton-opt) (.*?)[|\n]"
    matches = re.findall(pattern, text)
    if len(matches) < 1:
        return None
    if len(matches) > 1:
        #print(text)
        #raise ValueError("Expected at most one match")
        return None
    command, args = matches[0]
    return command, args.split()

cmd_mapping = dict()
for filename, command_strings in cmd_string_mapping.items():
    # concatenate command strings
    full_string = ""
    for cmd_str in command_strings:
        matches = re.findall(r"//\s+RUN:\s+(\S.*)", cmd_str)
        if len(matches) < 1:
            break
        if len(matches) > 1:
            raise ValueError("Expected at most one match")
        full_string += matches[0]
    full_string = full_string.replace("\\", "")
    #if "mlir-opt" in full_string:
    #    print(full_string)
    #    print("----")
    try:
        cmd_mapping[filename] = get_cmd(full_string)
    except:
        print(filename)

# filter out non mlir-opt commands
cmd_mapping = {file: cmdargs for file, cmdargs in cmd_mapping.items() if cmdargs is not None}
cmd_mapping

{'tritongpu_wmma_dot_to_llvm.mlir': ('triton-opt',
  ['%s', '--split-input-file', '--convert-triton-amdgpu-to-llvm']),
 'matmul.mlir': ('triton-opt',
  ['%s',
   '-split-input-file',
   '-convert-triton-to-tritongpu',
   '-tritongpu-remove-layout-conversions',
   '-tritongpu-pipeline=num-stages=3',
   '-canonicalize',
   '-test-print-allocation',
   '2>&1']),
 'fence-inserstion.mlir': ('triton-opt',
  ['%s', '-split-input-file', '--triton-nvidia-gpu-fence-insertion']),
 'coalesce.mlir': ('triton-opt',
  ['%s', '-split-input-file', '-tritongpu-coalesce']),
 'reorder-instructions.mlir': ('triton-opt',
  ['%s', '-split-input-file', '-tritongpu-reorder-instructions']),
 'combine.mlir': ('triton-opt',
  ['%s', '-split-input-file', '-canonicalize', '-triton-combine']),
 'rewrite-tensor-pointer.mlir': ('triton-opt',
  ['%s', '-triton-rewrite-tensor-pointer']),
 'triton_ops.mlir': ('triton-opt', ['%s']),
 'fp_to_fp.mlir': ('triton-opt',
  ['%s', '--split-input-file', '--convert-triton-amdgpu-t

In [4]:
dialects = [
    "tt",
    "triton_nvidia_gpu",
    "nvgpu",
    "triton_gpu",
]
# Triton doesn't follow the normal naming scheme for passes
dialects = ["triton"]

In [7]:
all_options = set()
dialect_associations = dict()
for cmd, args in cmd_mapping.values():
    for arg in args:
        all_options.add(arg)
        possible_match = re.match(r"--?convert-([a-z]+)-to", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect not in dialect_associations:
                dialect_associations[dialect] = []
            if arg not in dialect_associations[dialect]:
                dialect_associations[dialect].append(arg)
        possible_match = re.match(r"--?([a-z]+)", arg)
        if possible_match:
            dialect = possible_match.group(1)
            if dialect in dialects:
                if dialect not in dialect_associations:
                    dialect_associations[dialect] = []
                if arg not in dialect_associations[dialect]:
                    dialect_associations[dialect].append(arg)
with open("/workdir/mlir-eval/triton/dialect-associations.json", "w") as f:
    json.dump(dialect_associations, f)

In [6]:
dialect_associations

{'triton': ['-convert-triton-to-tritongpu',
  '--triton-nvidia-gpu-fence-insertion',
  '-triton-combine',
  '-triton-rewrite-tensor-pointer',
  '-triton-reorder-broadcast',
  '-convert-triton-to-tritongpu=num-warps=2'],
 'scf': ['--convert-scf-to-cf']}

In [8]:
all_options

{'%s',
 '%t',
 '--allocate-shared-memory',
 '--convert-nv-gpu-to-llvm',
 '--convert-scf-to-cf',
 '--convert-triton-amdgpu-to-llvm',
 '--convert-triton-gpu-to-llvm',
 '--convert-triton-gpu-to-llvm=compute-capability=70',
 '--convert-triton-gpu-to-llvm=compute-capability=90',
 '--cse',
 '--decompose-unsupported-conversions',
 '--dump-pass-pipeline',
 '--llvm-optimize-for-nvvm-target',
 '--mlir-disable-threading',
 '--run-reproducer',
 '--split-input-file',
 '--triton-nvidia-gpu-fence-insertion',
 '--tritonamdgpu-optimize-epilogue',
 '--tritongpu-accelerate-matmul=compute-capability=90',
 '--verify-diagnostics',
 '-canonicalize',
 '-convert-triton-to-tritongpu',
 '-convert-triton-to-tritongpu=num-warps=2',
 '-o',
 '-split-input-file',
 '-test-print-alias',
 '-test-print-alignment',
 '-test-print-allocation',
 '-test-print-membar',
 '-triton-combine',
 '-triton-reorder-broadcast',
 '-triton-rewrite-tensor-pointer',
 '-tritongpu-coalesce',
 '-tritongpu-optimize-dot-operands',
 '-tritongpu-o

In [9]:
actual_options = all_options - {
    "%s",
    "%t",
    "-o",
    "--verify-diagnostics",
    "--split-input-file",
    "--dump-pass-pipeline",
    "--mlir-disable-threading",
    "--run-reproducer",
    "-o",
    "-split-input-file",
    "-test-print-alias",
    "-test-print-alignment",
    "-test-print-allocation",
    "-test-print-membar",
    "-tritongpu-pipeline=compute-capability=90",
    "-tritongpu-pipeline=num-stages=3",
    "2>&1",
}
actual_options

{'--allocate-shared-memory',
 '--convert-nv-gpu-to-llvm',
 '--convert-scf-to-cf',
 '--convert-triton-amdgpu-to-llvm',
 '--convert-triton-gpu-to-llvm',
 '--convert-triton-gpu-to-llvm=compute-capability=70',
 '--convert-triton-gpu-to-llvm=compute-capability=90',
 '--cse',
 '--decompose-unsupported-conversions',
 '--llvm-optimize-for-nvvm-target',
 '--triton-nvidia-gpu-fence-insertion',
 '--tritonamdgpu-optimize-epilogue',
 '--tritongpu-accelerate-matmul=compute-capability=90',
 '-canonicalize',
 '-convert-triton-to-tritongpu',
 '-convert-triton-to-tritongpu=num-warps=2',
 '-triton-combine',
 '-triton-reorder-broadcast',
 '-triton-rewrite-tensor-pointer',
 '-tritongpu-coalesce',
 '-tritongpu-optimize-dot-operands',
 '-tritongpu-optimize-thread-locality',
 '-tritongpu-prefetch',
 '-tritongpu-remove-layout-conversions',
 '-tritongpu-reorder-instructions'}

In [10]:
with open("/workdir/mlir-eval/triton/dialect-associations.json", "w") as f:
    json.dump({"triton": list(actual_options)}, f)